# Dataset creation to train a YoloV8 model for tank detection

This notebook shows how to build a dataset of annotated images to train a computer vision model for object detection. We use available open source datasets to create a dataset of military vehicles and format it correctly for YoloV8 training.

We use [fiftyone](https://github.com/voxel51/fiftyone) to convert, merge, label and format the images prior to training with [Yolov8](https://github.com/ultralytics/ultralytics).

### Setup

We start by setting up some logging.

In [1]:
import logging

logging.basicConfig(level=logging.INFO)

### Download images from ImageNet

The first dataset we'll use is ImageNet21k. The ImageNet21k dataset is available at [https://image-net.org/download-images.php](https://image-net.org/download-images.php). You need to register and be granted access to download the images. We use the Winter 21 version since it gives the option of downloading the images for a single synset: https://image-net.org/data/winter21_whole/SYNSET_ID.tar, e.g., https://image-net.org/data/winter21_whole/n02352591.tar. The processed version of ImageNet21k is available here : https://github.com/Alibaba-MIIL/ImageNet21K. The class ids and names are available here https://github.com/google-research/big_transfer/issues/7#issuecomment-640048775.

We'll begin by downloading the class names that are in ImageNet21k and look for relevant classes that we can use.

In [2]:
from pathlib import Path

imagenet_dir = Path() / "imagenet"

In [3]:
from adomvi.datasets.imagenet import download_class_names, find_class_by_text

classes = download_class_names(imagenet_dir)
find_class_by_text(classes, "military")

INFO:root:File imagenet/imagenet21k_wordnet_ids.txt already exists. Skipping download.
INFO:root:File imagenet/imagenet21k_wordnet_lemmas.txt already exists. Skipping download.


{'n03762982': 'military_hospital',
 'n03763727': 'military_quarters',
 'n03763968': 'military_uniform',
 'n03764276': 'military_vehicle',
 'n04552348': 'warplane, military_plane',
 'n08249459': 'concert_band, military_band',
 'n09809538': 'army_engineer, military_engineer',
 'n09943239': 'commissioned_military_officer',
 'n10316360': 'military_attache',
 'n10316527': 'military_chaplain, padre, Holy_Joe, sky_pilot',
 'n10316862': 'military_leader',
 'n10317007': 'military_officer, officer',
 'n10317500': 'military_policeman, MP',
 'n10512372': 'recruit, military_recruit',
 'n10582746': 'serviceman, military_man, man, military_personnel',
 'n10759331': 'volunteer, military_volunteer, voluntary'}

We can now download images and annotations for the relevant classes. The `download_imagenet_detections` function will download the images and annotations for the given class ids **if the annotations exist** (not all classes have been annotated).

In [4]:
from adomvi.datasets.imagenet import download_imagenet_detections

class_ids = ["n02740300", "n04389033", "n02740533", "n04464852", "n03764276"]
download_imagenet_detections(class_ids, imagenet_dir)

INFO:root:File imagenet/bboxes_annotations.tar.gz already exists. Skipping download.
INFO:root:There are not annotations for class n02740300.
INFO:root:Annotations directory imagenet/labels/n04389033 already exists. Skipping extract.
INFO:root:There are not annotations for class n02740533.
INFO:root:There are not annotations for class n04464852.
INFO:root:There are not annotations for class n03764276.
INFO:root:Deleting annotations dir.


The data we just downloaded into the `imagenet` directory is not all clean: there are annotations which have no corresponding image. We need to remove those labels, otherwise this causes errors when importing the data into fiftyone.

In [5]:
from adomvi.datasets.imagenet import cleanup_labels_without_images

cleanup_labels_without_images(imagenet_dir)

INFO:root:Deleting 0 labels without images


We can now create a new dataset with `fiftyone`. Fiftyone allows us to manage images annotated with bounding boxes and labels, to merge datasets from different sources, and to split the datasets and prepare them for processing.

In [6]:
import fiftyone as fo

# Create the dataset
dataset = fo.Dataset.from_dir(
    dataset_dir=imagenet_dir,
    dataset_type=fo.types.VOCDetectionDataset,
)

# dataset.name = "military-vehicles"

dataset.map_labels(
    "ground_truth",
    {"n04389033":"tank"}
).save()

 100% |█████████████████| 378/378 [416.3ms elapsed, 0s remaining, 912.5 samples/s]      


INFO:eta.core.utils: 100% |█████████████████| 378/378 [416.3ms elapsed, 0s remaining, 912.5 samples/s]      


Once our dataset is created, we can launch a session to display the dataset and view the annotated images

In [7]:
session = fo.launch_app(dataset, auto=False)

Session launched. Run `session.show()` to open the App in a cell output.


INFO:fiftyone.core.session.session:Session launched. Run `session.show()` to open the App in a cell output.


In [8]:
session.open_tab()

<IPython.core.display.Javascript object>

In [9]:
session.freeze()

### Add OpenImage samples

The ImageNet dataset only contained 378 annotated images of tanks, so we'll look into other available datasets to improve training of the model. We’ll load [Open Images](https://storage.googleapis.com/openimages/web/index.html) samples with `Tank` detection labels, passing in `only_matching=True` to only load the `Tank` labels. We then map these labels by changing `Tank` into `tank`.

In [10]:
import fiftyone.zoo as foz

oi_samples = foz.load_zoo_dataset(
    "open-images-v7",
    classes = ["Tank"],
    only_matching=True,
    label_types="detections"
).map_labels(
    "ground_truth",
    {"Tank":"tank"}
)

INFO:fiftyone.zoo.datasets:Downloading split 'train' to '/home/ukemkata/fiftyone/open-images-v7/train' if necessary


Necessary images already downloaded


INFO:fiftyone.utils.openimages:Necessary images already downloaded


Existing download of split 'train' is sufficient


INFO:fiftyone.zoo.datasets:Existing download of split 'train' is sufficient


...

INFO:fiftyone.zoo.datasets:Loading 'open-images-v7' split 'validation'


 100% |███████████████████| 50/50 [88.4ms elapsed, 0s remaining, 565.8 samples/s]     


INFO:eta.core.utils: 100% |███████████████████| 50/50 [88.4ms elapsed, 0s remaining, 565.8 samples/s]     


Dataset 'open-images-v7' created


INFO:fiftyone.zoo.datasets:Dataset 'open-images-v7' created


In [11]:
dataset.merge_samples(oi_samples)

### Export dataset to disk

Now that our dataset is created, we'll export it into a format supported by YOLOv8 to train our model.

We first remove tags from the dataset, and split it into a train, val and test sets.

In [14]:
import fiftyone.utils.random as four

## delete existing tags to start fresh
dataset.untag_samples(dataset.distinct("tags"))

## split into train, test and val
four.random_split(dataset, {"train": 0.8, "val": 0.1, "test": 0.1})

Once our dataset is split, we can export it to a specific directory.

In [15]:
from pathlib import Path
from adomvi.yolo.utils import export_yolo_data

export_dir = Path() / "dataset"
export_yolo_data(dataset, export_dir, ["tank"], split = ["train", "val", "test"], overwrite=True)

 100% |███████████████| 1299/1299 [1.3s elapsed, 0s remaining, 1.0K samples/s]          


INFO:eta.core.utils: 100% |███████████████| 1299/1299 [1.3s elapsed, 0s remaining, 1.0K samples/s]          


Directory 'dataset' already exists; export will be merged with existing files


 100% |█████████████████| 163/163 [146.2ms elapsed, 0s remaining, 1.1K samples/s]     


INFO:eta.core.utils: 100% |█████████████████| 163/163 [146.2ms elapsed, 0s remaining, 1.1K samples/s]     


Directory 'dataset' already exists; export will be merged with existing files


 100% |█████████████████| 162/162 [164.9ms elapsed, 0s remaining, 982.2 samples/s]     


INFO:eta.core.utils: 100% |█████████████████| 162/162 [164.9ms elapsed, 0s remaining, 982.2 samples/s]     


### Add Roboflow dataset

The current data contained 1624 annotated images of tank, so we'll look into other available datasets to improve training of the model. We’ll load [Roboflow Images](https://universe.roboflow.com/) with `Tank` detection labels.

In [16]:
from roboflow import Roboflow
from pathlib import Path

roboflow_dir = Path() / "roboflow"
rf = Roboflow(api_key="qkW6N3pPNwzFNsQmD7Av")
project = rf.workspace("tank-detection-3r2du").project("tank-detection-gzcao")
version = project.version(1)
dataset_rf = version.download("yolov8", location=str(roboflow_dir))


loading Roboflow workspace...
loading Roboflow project...
Dependency ultralytics==8.0.196 is required but found version=8.2.31, to fix: `pip install ultralytics==8.0.196`


In [17]:
from adomvi.datasets.roboflow import copy_dataset_structure

copy_dataset_structure(roboflow_dir, export_dir)


Copied roboflow/test/images/22_jpg.rf.c9822cbe171e4a7769d0f55e001d6d8b.jpg to dataset/images/test
Copied roboflow/test/images/3_png.rf.f2ea8e8caa41bf8a5fb2637c4a9b6306.jpg to dataset/images/test
Copied roboflow/test/images/29_jpg.rf.ba857b24cd82e8db58d678a853c56b01.jpg to dataset/images/test
Copied roboflow/test/images/16_jpg.rf.ce92ea9affc34c2506dfa7a9e05da22c.jpg to dataset/images/test
Copied roboflow/test/images/14_jpg.rf.76fda5f717d5488e95cb5afda9a09416.jpg to dataset/images/test
Copied roboflow/test/images/21_jpg.rf.186c0b15c411bfcab48168531f415e9c.jpg to dataset/images/test
Copied roboflow/test/images/37_jpg.rf.bfc98f6264055141d918ed3dc3a2c6f7.jpg to dataset/images/test
Copied roboflow/test/images/53_jpg.rf.1cf122b2a00f0875de6abb6de5f2938e.jpg to dataset/images/test
Copied roboflow/test/images/18_jpg.rf.b62e1a478aefe8edff4dbb6702a72100.jpg to dataset/images/test
Copied roboflow/test/images/33_jpg.rf.571300a0641cd65819346c1ea1653ae9.jpg to dataset/images/test
Copied roboflow/test/